In [1]:
import torch
import sys
sys.path.append("/home/tuxae/MVA/recvis21_a3")

%load_ext autoreload
%autoreload 2

from torch import nn
from torchsummary import summary
import torchvision.models as models
import torch.optim as optim

# Data initialization and loading
from src.data import data_transforms
from torchvision import datasets

from PIL import Image
from torchvision import transforms

import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np

from tqdm.notebook import tqdm_notebook

import time
import copy

In [2]:
BATCH_SIZE = 64
DATA_PATH = "../299_cropped_bird_dataset"
DATA_TRANSFORMS = transforms.Compose(
    [
        transforms.transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
EMBEDDINGS_PATH = "../embeddings"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

MODEL = models.inception_v3(pretrained=True)
for param in MODEL.parameters():
    param.requires_grad = False
    
# Parameters of newly constructed modules have requires_grad=True by default
# Handle the auxilary net
num_ftrs = MODEL.AuxLogits.fc.in_features
MODEL.AuxLogits.fc = nn.Linear(num_ftrs, 20)
# Handle the primary net
num_ftrs = MODEL.fc.in_features
MODEL.fc = nn.Linear(num_ftrs, 20)

In [3]:
summary(MODEL, input_size=(3, 299, 299))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             864
       BatchNorm2d-2         [-1, 32, 149, 149]              64
       BasicConv2d-3         [-1, 32, 149, 149]               0
            Conv2d-4         [-1, 32, 147, 147]           9,216
       BatchNorm2d-5         [-1, 32, 147, 147]              64
       BasicConv2d-6         [-1, 32, 147, 147]               0
            Conv2d-7         [-1, 64, 147, 147]          18,432
       BatchNorm2d-8         [-1, 64, 147, 147]             128
       BasicConv2d-9         [-1, 64, 147, 147]               0
        MaxPool2d-10           [-1, 64, 73, 73]               0
           Conv2d-11           [-1, 80, 73, 73]           5,120
      BatchNorm2d-12           [-1, 80, 73, 73]             160
      BasicConv2d-13           [-1, 80, 73, 73]               0
           Conv2d-14          [-1, 192,

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(DATA_PATH + "/train_images", transform=DATA_TRANSFORMS),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(DATA_PATH + "/val_images", transform=DATA_TRANSFORMS),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
)


In [5]:
import argparse
import os

class Args:
    data = "../299_cropped_bird_dataset"
    batch_size = BATCH_SIZE
    epochs = 20
    lr = 0.01
    momentum = 0.9
    scheduler_step = 5
    gamma = 0.1
    seed = 1
    log_interval = 10
    experiment = "../inceptionv3_experiment"
    
args = Args()
use_cuda = torch.cuda.is_available()
torch.manual_seed(args.seed)

# Create experiment folder
if not os.path.isdir(args.experiment):
    os.makedirs(args.experiment)

In [6]:
from torch.optim import lr_scheduler

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = MODEL.to(device)

# Observe that only parameters of final layer are being optimized as
# opposed to before.
params_to_update = []
for name,param in MODEL.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

# optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(
     params_to_update,
     lr=0.001,
     weight_decay=3e-4,
)

criterion = nn.CrossEntropyLoss(reduction="mean")

# Decay LR by a factor of 0.1 every epoch
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [7]:
def train(epoch):
    MODEL.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = MODEL(data)
        
        outputs, aux_outputs = MODEL(data)
        loss1 = criterion(outputs, target)
        loss2 = criterion(aux_outputs, target)
        loss = loss1 + 0.4*loss2

        loss.backward()
        optimizer.step()
        scheduler.step()
        if batch_idx % args.log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.data.item(),
                )
            )


def validation():
    MODEL.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        outputs = MODEL(data)
        # sum up batch loss
        criterion = torch.nn.CrossEntropyLoss(reduction="mean")
        validation_loss += criterion(outputs, target).data.item()
        # get the index of the max log-probability
        
        #
        _, pred = torch.max(outputs, 1)
        
        #_, pred = outputs.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print(
        "\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)".format(
            validation_loss,
            correct,
            len(val_loader.dataset),
            100.0 * correct / len(val_loader.dataset),
        )
    )

In [8]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    validation()

    # train_model(model, optimizer, scheduler, num_epochs=args.epochs)

    model_file = args.experiment + "/inceptionv3_model_" + str(epoch) + ".pth"
    torch.save(MODEL.state_dict(), model_file)
    print(
        "Saved model to "
        + model_file
        + ". You can run `python evaluate.py --model "
        + model_file
        + "` to generate the Kaggle formatted csv file\n"
    )

Train Epoch: 1 [0/1082 (0%)]	Loss: 4.203300
Train Epoch: 1 [640/1082 (59%)]	Loss: 4.040770

Validation set: Average loss: 0.0544, Accuracy: 22/103 (21%)
Saved model to ../inceptionv3_experiment/inceptionv3_model_1.pth. You can run `python evaluate.py --model ../inceptionv3_experiment/inceptionv3_model_1.pth` to generate the Kaggle formatted csv file

Train Epoch: 2 [0/1082 (0%)]	Loss: 3.928830
Train Epoch: 2 [640/1082 (59%)]	Loss: 3.887494

Validation set: Average loss: 0.0547, Accuracy: 19/103 (18%)
Saved model to ../inceptionv3_experiment/inceptionv3_model_2.pth. You can run `python evaluate.py --model ../inceptionv3_experiment/inceptionv3_model_2.pth` to generate the Kaggle formatted csv file

Train Epoch: 3 [0/1082 (0%)]	Loss: 3.880659
Train Epoch: 3 [640/1082 (59%)]	Loss: 3.909591

Validation set: Average loss: 0.0546, Accuracy: 20/103 (19%)
Saved model to ../inceptionv3_experiment/inceptionv3_model_3.pth. You can run `python evaluate.py --model ../inceptionv3_experiment/inception

KeyboardInterrupt: 